In [32]:
using Plots;
pythonplot();
using LinearAlgebra;
using JLD2;
using LaTeXStrings;

include("../../Hamiltonians/H2.jl");
using .H2;
include("../../Hamiltonians/H4.jl");
using .H4;
include("../../Helpers/ChaosIndicators.jl");
using .ChaosIndicators;

colors = [:dodgerblue :darkviolet :limegreen :indianred :magenta :darkblue :aqua :deeppink :dimgray :red :royalblue :slategray :black :lightseagreen :forestgreen :palevioletred]
markers = ["o","x","v","*","H","D","s","d","P","2","|","<",">","_","+",","]
markers_line = ["-o","-x","-v","-H","-D","-s","-d","-P","-2","-|","-<","->","-_","-+","-,"]


15-element Vector{String}:
 "-o"
 "-x"
 "-v"
 "-H"
 "-D"
 "-s"
 "-d"
 "-P"
 "-2"
 "-|"
 "-<"
 "->"
 "-_"
 "-+"
 "-,"

Level Spacing Ratio:

In [25]:
function GetLevelSpacingRatioData(L′s:: Vector{Int}, q′s:: Vector{Float64}, maxNumbOfIter:: Int64)
    r = Vector{Vector{Float64}}(undef, length(L′s))

    for (i,L) in enumerate(L′s)
        println(L, "--------")
        r[i] = ChaosIndicators.LevelSpacingRatio(L, q′s, maxNumbOfIter)

        folder = jldopen("./Data-ChaosIndicators/LevelSpacingRatio_L$(L)_Iter$(maxNumbOfIter).jld2", "w");
        folder["r"] = r[i];
        folder["q′s"] = q′s;
        close(folder)        
    end

    println(r)

end


GetLevelSpacingRatioData (generic function with 1 method)

In [27]:
function PlotLevelSpacingRatio(L′s:: Vector{Int}, q′s:: Vector{Float64}, maxNumbOfIter:: Int64)

    levelSpacingRatio = plot(xscale=:log10, minorgrid=true, framestyle = :box, legend=:topleft);

    plot!(levelSpacingRatio, q′s, 0.3863 .* ones(Float64, length(q′s)), ls=:dash, label="Poisson");
    plot!(levelSpacingRatio, q′s, 0.5359 .* ones(Float64, length(q′s)), ls=:dash, label="GOE");

    for (i,L) in enumerate(L′s) 
        folder = jldopen("./Data-ChaosIndicators/LevelSpacingRatio_L$(L)_Iter$(maxNumbOfIter).jld2", "r");
        r = folder["r"];
        q′s = folder["q′s"];
        close(folder);

        plot!(levelSpacingRatio, q′s, r, lc = colors[i], label="L=$(L)");

    end

    xlabel!(L"q");
    ylabel!(L"r");

    return levelSpacingRatio;
end

PlotLevelSpacingRatio (generic function with 1 method)

Informational entropy

In [28]:
function GetInformationalEntropyData(L′s:: Vector{Int}, q′s:: Vector{Float64}, maxNumbOfIter:: Int64)
    S = Vector{Vector{Float64}}(undef, length(L′s))

    for (i,L) in enumerate(L′s)
        println(L, "--------")
        S[i] = ChaosIndicators.InformationalEntropy(L, q′s, maxNumbOfIter)

        folder = jldopen("./Data-ChaosIndicators/InformationalEntropy_L$(L)_Iter$(maxNumbOfIter)_fixedNormalization.jld2", "w");
        folder["S"] = S[i];
        folder["q′s"] = q′s;
        close(folder)        
    end

    println(S)

end

GetInformationalEntropyData (generic function with 1 method)

In [29]:
function PlotInformationalEntropy(L′s:: Vector{Int}, q′s:: Vector{Float64}, maxNumbOfIter:: Int64)
    informationalEntropy = plot(xscale=:log10, minorgrid=true, framestyle = :box, legend=:topleft);

    plot!(informationalEntropy, q′s,  ones(Float64, length(q′s)), ls=:dash, label="");

    for (i,L) in enumerate(L′s) 
        folder = jldopen("./Data-ChaosIndicators/InformationalEntropy_L$(L)_Iter$(maxNumbOfIter).jld2", "r");
        S = folder["S"];
        q′s = folder["q′s"];
        close(folder);
        
        plot!(informationalEntropy, q′s, S, lc = colors[i], label="L=$(L)");

    end

    xlabel!(L"q");
    ylabel!(L"S");

    return informationalEntropy;
end

PlotInformationalEntropy (generic function with 1 method)

Entanglement Entropy

In [33]:
function GetEntanglementEntropyData(L′s:: Vector{Int}, q′s:: Vector{Float64}, maxNumbOfIter:: Int64)

    for (i,L) in enumerate(L′s)
        println(L, "--------");
        permutation1 = [1:1:L...];
        permutation2 = [1:2:L..., 2:2:L...];
        permutations = [permutation1, permutation2];

        E = ChaosIndicators.EntanglementEntropy(L, q′s, maxNumbOfIter, permutations)

        folder = jldopen("./Data-ChaosIndicators/EntanglementEntropy_L$(L)_Iter$(maxNumbOfIter).jld2", "w");
        folder["E"] = E;
        folder["q′s"] = q′s;
        close(folder)        
    end


end

GetEntanglementEntropyData (generic function with 1 method)

Retrive data

In [9]:
L′s = [6,8,10,12];
x= LinRange(-5, 1, 20);
q′s = 10 .^(x);
maxNumbOfIter = 100

# GetLevelSpacingRatioData(L′s, q′s, maxNumbOfIter)

100

In [10]:
L′s = [6,8,10,12];
x= LinRange(-5, 1, 20);
q′s = 10 .^(x);
maxNumbOfIter = 200

# GetInformationalEntropyData(L′s, q′s, maxNumbOfIter)

200

In [34]:
L′s = [12];
x= LinRange(-3, 1, 20);
q′s = 10 .^(x);
maxNumbOfIter = 200

GetEntanglementEntropyData(L′s, q′s, maxNumbOfIter);

12--------


In [12]:
function JoinTwoLSRFiles(L′s, maxNumbOfIter1, maxNumbOfIter2)
    
    for L in L′s
        folder1 = jldopen("./Data/LevelSpacingRatio_L$(L)_Iter$(maxNumbOfIter1).jld2", "r");
        r1 = folder1["r"];
        q′s1 = folder1["q′s"];
        close(folder1);
        
        folder2 = jldopen("./Data/LevelSpacingRatio_L$(L)_Iter$(maxNumbOfIter2).jld2", "r");
        r2 = folder2["r"];
        q′s2 = folder2["q′s"];
        close(folder2);

        q′s1 == q′s2 ? println("q-ji so ok") : println("q-ji niso ok");

        maxNumbOfIter = maxNumbOfIter1 + maxNumbOfIter2;
        r = (r1*maxNumbOfIter1 + r2*maxNumbOfIter2)/maxNumbOfIter;

        folder = jldopen("./Data/LevelSpacingRatio_L$(L)_Iter$(maxNumbOfIter).jld2", "w");
        folder["r"] = r;
        folder["q′s"] = q′s;
        close(folder);
    end
end

# L′s = [6, 8, 10, 12];
# maxNumbOfIter1 = 400;
# maxNumbOfIter2 = 100;

# JoinTwoLSRFiles(L′s, maxNumbOfIter1, maxNumbOfIter2)

JoinTwoLSRFiles (generic function with 1 method)

Plotting data

In [19]:
function Plot1(L′s_lsr:: Vector{Int}, L′s_ie:: Vector{Int}, q′s_lsr:: Vector{Float64}, q′s_ie:: Vector{Float64}, maxNumbOfIter_lsr:: Int64, maxNumbOfIter_ie:: Int64)

    levelSpacingRatio = PlotLevelSpacingRatio(L′s_lsr, q′s_lsr, maxNumbOfIter_lsr);
    informationalEntropy = PlotInformationalEntropy(L′s_ie, q′s_ie, maxNumbOfIter_ie);

    ratio = 2.3;
    dim = 500;
    p = plot(levelSpacingRatio, informationalEntropy, layout=(1,2), size=(Int(dim*ratio),dim));

    savefig(p, "./Images-ChaosIndicators/ChaosIndicators_L$(L′s_lsr)_$(maxNumbOfIter_lsr)_$(maxNumbOfIter_lsr).pdf");
end

Plot1 (generic function with 1 method)

In [23]:
L′s_lsr = [6,8,10,12];
maxNumbOfIter_lsr = 500;
x_lsr= LinRange(-5, 1, 10);
q′s_lsr = 10 .^(x);

L′s_ie  = [8,10,12];
maxNumbOfIter_ie = 200;
x_ie= LinRange(-5, 1, 10);
q′s_ie = 10 .^(x);


levelSpacingRatio = PlotLevelSpacingRatio(L′s_lsr, q′s_lsr, maxNumbOfIter_lsr);
ratio = 1.4;
dim = 300;
p = plot(levelSpacingRatio, layout=(1,1), size=(Int(dim*ratio),dim));
savefig(p, "./Images-ChaosIndicators/SpektralnaStatistikaHamiltonke_L$(L′s_lsr)_$(maxNumbOfIter_lsr)_$(maxNumbOfIter_lsr).pdf");


informationalEntropy = PlotInformationalEntropy(L′s_ie, q′s_ie, maxNumbOfIter_ie);

# Plot1(L′s_lsr, L′s_ie, q′s_lsr, q′s_ie, maxNumbOfIter_lsr, maxNumbOfIter_ie)

┌ Warning: Opening file with JLD2.MmapIO failed, falling back to IOStream
└ @ JLD2 /Users/rokpintar/.julia/packages/JLD2/r5t7Q/src/JLD2.jl:286


SystemError: SystemError: opening file "./Data-ChaosIndicators/InformationalEntropy_L8_Iter200.jld2": No such file or directory

In [17]:
function PlotEntanglementEntropy(L′s:: Vector{Int},  maxNumbOfIter′s:: Vector{Int})
    compactBipartion    = plot(xscale=:log10, minorgrid=true, framestyle = :box, legend=:topleft, title = "Prepletenostna entropija"); 
    nonCompactBipartion = plot(xscale=:log10, minorgrid=true, framestyle = :box, legend=:topleft, title = "Prepletenostna entropija"); 

    # plot!(informationalEntropy, q′s,  ones(Float64, length(q′s)), ls=:dash, label="");

    for (i,L) in enumerate(L′s) 
        # D = binomial(L,L÷2)
        # D=Int(2^L)÷2
        # norm = log(D/2)
        norm = 1.

        folder = jldopen("./Data-ChaosIndicators/EntanglementEntropy_L$(L)_Iter$(maxNumbOfIter′s[i]).jld2", "r");
        E = folder["E"]./norm;
        q′s = folder["q′s"];
        close(folder);
        
        plot!(compactBipartion, q′s, E[1,:], lc = colors[i], label="L=$(L)");
        plot!(nonCompactBipartion, q′s, E[2,:], lc = colors[i], label="L=$(L)");
    end

    xlabel!(compactBipartion, L"q");
    ylabel!(compactBipartion, L"E");

    xlabel!(nonCompactBipartion, L"q");
    ylabel!(nonCompactBipartion, L"E");

    ratio = 2.3;
    dim = 500;
    p = plot(compactBipartion, nonCompactBipartion, layout=(1,2), size=(Int(dim*ratio),dim));

    savefig(p, "./Images-ChaosIndicators/EntanglementEntropy_L$(L′s)_$(maxNumbOfIter).pdf");
    
end

PlotEntanglementEntropy (generic function with 2 methods)

In [23]:
L′s = [6,8];
maxNumbOfIter′s = [200, 200]

PlotEntanglementEntropy(L′s, maxNumbOfIter′s)

"/Users/rokpintar/Desktop/FAX/MAG 2. /Magisterska naloga/MastersDegree-Program/Plotting/ChaosIndicators/Images-ChaosIndicators/EntanglementEntropy_L[6, 8]_200.pdf"

In [48]:
using PyPlot

function PlotEntanglementEntropyInPyPlot(L′s:: Vector{Int}, maxNumbOfIter′s:: Vector{Int}, q′s:: Vector{Float64})
    plt.style.use(["don_custom"])
    fig, ax = plt.subplots(ncols=2, figsize=(9, 4))


    for (i,L) in enumerate(L′s) 
        # D = binomial(L,L÷2)
        # D=Int(2^L)÷2
        # norm = log(D/2)
        # norm = 1.

        folder = jldopen("./Data-ChaosIndicators/EntanglementEntropy_L$(L)_Iter$(maxNumbOfIter′s[i]).jld2", "r");
        E = folder["E"]./norm;
        q′s = folder["q′s"];
        close(folder);

        ax[1].plot(q′s, E[1,:], label=L"L=%$(L)")
        ax[2].plot(q′s, E[2,:], label=L"L=%$(L)")
    end

    ax[1].legend()
    ax[2].legend()

    fig.tight_layout()
    display(fig)

end




In [ ]:
L′s = [6,8,10];
maxNumbOfIter′s = [200, 200, 200]

PlotEntanglementEntropyInPyPlot(L′s, maxNumbOfIter′s, q′s)